In [ ]:
import gymnasium as gym
import numpy as np

In [ ]:
class Board(gym.Env):
    def __init__(self):
        pass

    def step(self, action):
        pass

    def reset(self):
        pass

    def render(self, mode='human'):
        pass

In [12]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.dqn import DQNConfig
from ray.tune.logger import pretty_print

# config = PPOConfig()
config = DQNConfig()
print(config)

algo = config.resources(num_gpus=0).environment("CartPole-v0").build()

for i in range(10):
    result = algo.train()
    print(pretty_print(result))

2023-08-01 15:01:16,210	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!


2023-08-01 15:01:16,219	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
2023-08-01 15:01:16,224	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/simple_q/` has been deprecated. Use `rllib_contrib/simple_q/` instead. This will raise an error in the future!
c:\Users\lcglab\miniconda3\envs\rl\lib\site-packages\gymnasium\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
2023-08-01 15:01:16,272	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
2023-08-01 15:01:16,274	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.torch_modelv2.TorchModelV2` has been deprecated. Use `ray.rllib.core.rl_module.rl_module.RLModule` instead. This will raise an error in the futur

2023-08-01 15:01:20,599	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


agent_timesteps_total: 1000
connector_metrics:
  ObsPreprocessorConnector_ms: 0.01042534907658895
  StateBufferConnector_ms: 0.014575322469075521
  ViewRequirementAgentConnector_ms: 0.15203158060709634
counters:
  num_agent_steps_sampled: 1000
  num_agent_steps_trained: 0
  num_env_steps_sampled: 1000
  num_env_steps_trained: 0
custom_metrics: {}
date: 2023-08-01_15-01-20
done: false
episode_len_mean: 20.583333333333332
episode_media: {}
episode_reward_max: 62.0
episode_reward_mean: 20.583333333333332
episode_reward_min: 9.0
episodes_this_iter: 48
episodes_total: 48
hostname: DESKTOP-TSBOB2F
info:
  learner: {}
  num_agent_steps_sampled: 1000
  num_agent_steps_trained: 0
  num_env_steps_sampled: 1000
  num_env_steps_trained: 0
iterations_since_restore: 1
node_ip: 127.0.0.1
num_agent_steps_sampled: 1000
num_agent_steps_trained: 0
num_env_steps_sampled: 1000
num_env_steps_sampled_this_iter: 1000
num_env_steps_sampled_throughput_per_sec: 239.314557840487
num_env_steps_trained: 0
num_env_s

In [10]:
algo.export_model('model')

{'model': 'C:\\Users\\lcglab/ray_results\\PPO_CartPole-v0_2023-08-01_14-15-57v3dx_mi1\\model'}

In [15]:
algo.export_policy_model("policy_model", onnx=18)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [16]:
config.get("_enable_rl_module_api", False)

False

In [17]:
import onnxruntime
onnxruntime.__version__

'1.15.1'